In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.layers import Layer, Dropout, GRU, LSTM
import pywt
import cufflinks as cf
cf.go_offline()

In [2]:
class AttentionMechanism(Layer):
    def __init__(self, return_sequences=True):
        super(AttentionMechanism, self).__init__()
        self.return_sequences = return_sequences

    def build(self, input_shape):
        self.W=self.add_weight(name="att_weight", shape=(input_shape[-1],1),
                               initializer="normal")
        self.b=self.add_weight(name="att_bias", shape=(input_shape[1],1),
                               initializer="zeros")

    def call(self, x):
        e = tf.nn.tanh(tf.matmul(x,self.W)+self.b)
        a = tf.nn.softmax(e, axis=1)
        output = x*a

        if self.return_sequences:
            return output

        return tf.reduce_sum(output, axis=1)

In [3]:
class PreprocessData:
    def __init__(self, file_path, feature_column, look_back=2):
        self.file_path = file_path
        self.feature_column = feature_column
        self.look_back = look_back
        self.scaler = MinMaxScaler(feature_range=(0, 1))
        self.data_scaled = None
        self.X = None
        self.y = None

    def check_look_back(self):
        if self.look_back == 0:
            raise ValueError('You can not use number 0')

    def load_preprocess_data(self):
        df = pd.read_csv(self.file_path, index_col=['date'], parse_dates=True)
        df[self.feature_column].fillna(method='ffill', inplace=True)
        self.data_scaled = self.scaler.fit_transform(df[[self.feature_column]])

    def create_dataset(self):
        X, y = [], []
        
        for i in range(len(self.data_scaled) - self.look_back):
            a = self.data_scaled[i:(i + self.look_back), 0]
            X.append(a)
            y.append(self.data_scaled[i + self.look_back, 0])
        
        self.X, self.y = np.array(X), np.array(y)

        X_dif = self.X[:-1] - self.X[1:]
        X_new = np.hstack([self.X[:-1], X_dif])
        self.y = self.y[:-1]

        self.X = np.reshape(X_new, (X_new.shape[0], X_new.shape[1], 1))

    def split_data(self, train_size=0.8):
        split_idx = int(len(self.X) * train_size)
        X_train, X_test = self.X[:split_idx], self.X[split_idx:]
        y_train, y_test = self.y[:split_idx], self.y[split_idx:]
        
        return X_train, X_test, y_train, y_test

In [4]:
class TrainModel:
    def __init__(self):
        self.model = None

    def build_model_simple_rnn(self, input_shape):
        self.model = Sequential([
            SimpleRNN(50, activation='relu', input_shape=input_shape, return_sequences=True),
            SimpleRNN(50, activation='relu', input_shape=input_shape, return_sequences=True),
            AttentionMechanism(return_sequences=True),
            SimpleRNN(50, activation='relu'),
            Dense(1)
        ])
        self.model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

        return self.model

    def train_model(self, X_train, y_train, epochs=100, batch_size=64):
        self.model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2)

    def evaluate_model(self, X_test, y_test, scaler):
        predictions = self.model.predict(X_test)
        predictions = scaler.inverse_transform(predictions)
        y_test_actual = scaler.inverse_transform(y_test.reshape(-1, 1))
        
        mape = np.mean(np.abs((y_test_actual - predictions) / y_test_actual)) * 100
        mae = np.mean(np.abs(y_test_actual - predictions))
        
        print(f"MAPE: {mape}%")
        print(f"MAE: {mae}")
        
        return predictions, y_test_actual

    def plot_results(self, predictions, y_test_actual, feature_column):
        df = pd.DataFrame({
            'Actual': y_test_actual.ravel(),
            'Predicted': predictions.ravel()
        })
        
        df.iplot(title=f"Прогноз vs Реальные значения (Тест) для {feature_column}",
                 xTitle="Время",
                 yTitle="Значение",
                 theme="solar")

In [17]:
class ValidationData:
    def __init__(self, feature_column, scaler, look_back=1):
        self.feature_column = feature_column
        self.look_back = look_back
        self.scaler = scaler
        self.model = None
        self.X_val = None
        self.y_val = None
        self.predictions_val = None
        self.y_val_actual = None

    def check_look_back(self):
        if self.look_back == 0:
            raise ValueError('You can not use number 0')

    def load_validation_data(self, file_path, model):
        try:
            self.scaler = MinMaxScaler()
            df = pd.read_csv(file_path, index_col=['date'], parse_dates=True)
            df[self.feature_column].fillna(method='ffill', inplace=True)
            self.data_scaled = self.scaler.fit_transform(df[[self.feature_column]])

            X, y = [], []
            
            for i in range(len(self.data_scaled) - self.look_back):
                a = self.data_scaled[i:(i + self.look_back), 0]
                X.append(a)
                y.append(self.data_scaled[i + self.look_back, 0])
            
            self.X, self.y = np.array(X), np.array(y)

            X_dif = self.X[:-1] - self.X[1:]
            X_new = np.hstack([self.X[:-1], X_dif])
            self.y = self.y[:-1]

            self.X = np.reshape(X_new, (X_new.shape[0], X_new.shape[1], 1))
    
            predictions = model.predict(self.X)
            predictions = self.scaler.inverse_transform(predictions)
            y_actual = self.scaler.inverse_transform(self.y.reshape(-1, 1))

            return predictions, y_actual
            
        except Exception as e:
            print(f"Error in load_validation_data: {str(e)}")


    def plot_val_results(self):
        if self.predictions_val is None or self.y_val_actual is None:
            print("Predictions not generated. Please call generate_predictions first.")
            return

        df_val = pd.DataFrame({
            'Actual (Validation)': self.y_val_actual.ravel(),
            'Predicted (Validation)': self.predictions_val.ravel()
        })

        df_val.iplot(title=f"Forecast vs Actual Values (Validation) for {self.feature_column}",
                     xTitle="Time",
                     yTitle="Value",
                     theme="solar")

In [12]:
# Преобразование данных для обучения модели
preprocess_meantemp = PreprocessData('DailyDelhiClimateTrain.csv', 'meantemp', look_back=5)
preprocess_meantemp.check_look_back()
preprocess_meantemp.load_preprocess_data()
preprocess_meantemp.create_dataset()
X_train_meantemp, X_test_meantemp, y_train_meantemp, y_test_meantemp = preprocess_meantemp.split_data()

In [13]:
# Тренировка модели
trainer_meantemp = TrainModel()
trainer_meantemp.build_model_simple_rnn(input_shape=(X_train_meantemp.shape[1], 1))
trainer_meantemp.train_model(X_train_meantemp, y_train_meantemp)

Epoch 1/100
15/15 [==============================] - 2s 24ms/step - loss: 0.4788 - val_loss: 0.1928
Epoch 2/100
15/15 [==============================] - 0s 6ms/step - loss: 0.1366 - val_loss: 0.0455
Epoch 3/100
15/15 [==============================] - 0s 6ms/step - loss: 0.0524 - val_loss: 0.0410
Epoch 4/100
15/15 [==============================] - 0s 8ms/step - loss: 0.0464 - val_loss: 0.0160
Epoch 5/100
15/15 [==============================] - 0s 7ms/step - loss: 0.0095 - val_loss: 0.0040
Epoch 6/100
15/15 [==============================] - 0s 7ms/step - loss: 0.0040 - val_loss: 0.0019
Epoch 7/100
15/15 [==============================] - 0s 7ms/step - loss: 0.0029 - val_loss: 0.0017
Epoch 8/100
15/15 [==============================] - 0s 7ms/step - loss: 0.0024 - val_loss: 0.0016
Epoch 9/100
15/15 [==============================] - 0s 6ms/step - loss: 0.0022 - val_loss: 0.0020
Epoch 10/100
15/15 [==============================] - 0s 6ms/step - loss: 0.0023 - val_loss: 0.0018
Epoch 11

In [14]:
# Визуализация результатов обучения модели
predictions_meantemp, y_test_actual_meantemp = trainer_meantemp.evaluate_model(X_test_meantemp, y_test_meantemp, preprocess_meantemp.scaler)
trainer_meantemp.plot_results(predictions_meantemp, y_test_actual_meantemp, preprocess_meantemp.feature_column)

10/10 [==============================] - 1s 5ms/step
MAPE: 0.7812692376373458%
MAE: 0.23804940809448769


In [20]:
validation_data_meantemo = ValidationData(preprocess_meantemp.feature_column, preprocess_meantemp.look_back, )
validation_data_meantemo.load_validation_data('DailyDelhiClimateTest.csv', trainer_meantemp.model)

Error in load_validation_data: unsupported operand type(s) for -: 'int' and 'MinMaxScaler'
